# 🍏 Welcome to the **Smart Droplets Hackathon**!

![image](assets/sd_logo.png)

The Smart Droplets project is an EU-funded project, focusing on achieving reduced pesticide and fertilizer use with techniques of Digital Twins and Reinforcement Learning. One of the pilot projects of Smart Droplets is about the reduction of the **apple scab** pest.
In commercial apple production, **apple scab (_Venturia inaequalis_)** is the most economically important disease. Growers traditionally rely on **calendar-based fungicide programs**, which can lead to unnecessary sprays, resistance, and environmental impact.
In this hackathon we’ll flip that paradigm: **you will train a reinforcement-learning (RL) agent, or an intelligent conditional agent, to decide _when_ (and _how much_) to spray, balancing disease risk with sustainability.**

---

## 🧩 The Challenge

**Goal**: **Learn an optimal and adaptive spraying policy.**

**How to achieve that?**: Use **RL** to interact with the **A-scab** disease simulator. Your agent chooses daily actions (“spray how much” vs. “don’t spray”) through an entire season, to reduce the risk of breakout.

**Why is this important?** Smarter timing reduces chemical use, lowers costs, and lowers environmental impact while keeping orchards healthy to reduce yield loss.

Success is measured by a **cumulative reward**:

$R_t = - Risk_{t} - \beta P_t$

* $Risk$ is **Infection risk** – cumulative infection severity at harvest
* $\beta$ is **trade-off coefficient** - coefficient describing economic and ecological price of pesticides
* $P$ is **Pesticide amount** – the amount of pesticide sprayed

A higher score is better -- closer to zero!

---

## 🌱 The Environment: **Ascabgym**

Ascabgym is a stochastic, weather-driven model of apple scab dynamics, based on the A-scab model.
At each daily time step, an agent can observe the following features:

* _Weather features_: Temperature, relative humidity, rainfall, and leaf wetness
    * also 1 day and 2 day forecasts of the above weather features
* _Pest features_: ascopore maturity & development
* _Tree features_: leaf area index and phenology

…and updates disease progress based on your action. We expose a **Gymnasium-style API** so you can plug in any RL library (Stable-Baselines3, RLlib, CleanRL, etc.).

---

## 📂 Data & Starter Kit

* **Historical weather files** for training the model, files from several locations in Europe (CSV).
* **Default orchard parameters** which are already embedded in the model
* Performance of baseline agents to beat!

% Clone the repo or fork the Kaggle notebook, pip-install extra libraries as needed, and start experimenting.

---

## 🏆 Evaluation & Leaderboards

1. **Local validation** – run episodes on the provided weather years (fast iteration).
2. **Public leaderboard** – on submission, Kaggle simulates **four hidden seasons** from a particular location and reveals your average score.
3. **Private leaderboard** (final ranking) – after the deadline we evaluate on **five additional unseen seasons** to prevent overfitting.

> **Tip:** Aim for generalisation, not leaderboard-hacking!

---

## ✋ Rules of the Game

| Topic | Rule                                                                                               |
|-------|----------------------------------------------------------------------------------------------------|
| **External data** | In general, the use of external data is not allowed.                                               |
| **Compute** | Submissions must run < **3 hours on Kaggle’s 2×V100** quota.                                       |
| **Team size** | Individual!                                                                                        |
| **Fair play** | No model-sharing between teams until after the hackathon closes. Respect Kaggle’s Code of Conduct. |

---

## 🛠️ Resources

* **A-scab docs** (PDF + API reference)
* **RL documentation** (Stable Baselines 3 documentation)

---


## 🚀 First things first

Install all the necessary packages. Feel free to install the RL framework you feel most comfortable working with. By default, you can use Stable Baselines 3. Make sure to install the hackathon version of the A-scab repository.

In [ ]:
# install Ascabgym, note the @hackathon at the end of the repo link.
!pip install ascab@git+https://github.com/BigDataWUR/A-scab.git@hackathon

#clone the repository
!git clone -b hackathon https://github.com/WUR-AI/A-scab.git
#change directory
%cd A-scab
#install poetry if needed.
!pip install -qqq poetry
#install; this step may take 5 minutes
!poetry install --quiet

In [ ]:
# install further packages using poetry, to keep package dependency intact
# in general the syntax is `poetry add PACKAGE_NAME`, then install with `poetry install`.
# below is an example how to install stable baselines 3

!poetry add stable-baselines3
!poetry install

Initialize the gymnasium environment by importing the necessary methods:

In [ ]:
import os
from ascab.env.env import MultipleWeatherASCabEnv, get_weather_library_from_csv, ActionConstrainer

The code below will construct the gym environment!

In [ ]:
ascab_train = MultipleWeatherASCabEnv(
    weather_data_library=get_weather_library_from_csv(os.path.join('content', 'dataset', 'train.csv')),
    discrete_actions=True,  # Feel free to change this if using a method or RL agent that uses continuous actions.
)


The code below will be your validation gym environment, i.e., the place where you test your trained agents!

In [ ]:
ascab_val = MultipleWeatherASCabEnv(
    weather_data_library=get_weather_library_from_csv(os.path.join('content', 'dataset', 'train.csv')),
    discrete_actions=True,  # Feel free to change this if using a method or RL agent that uses continuous actions.
)

# Example of training your own agent

### Training with Stable Baselines 3

Below we provide an example of training your RL-based pesticide expert with the DQN algorithm, provided by the Stable Baselines 3 algorithm.

In [ ]:
from ascab.train import RLAgent
from stable_baselines3 import DQN

os.makedirs(os.path.join(os.getcwd(), 'rl_model'), exist_ok=True)
os.makedirs(os.path.join(os.getcwd(), 'log'), exist_ok=True)

rl_agent = RLAgent(
    ascab_train=ascab_train,
    ascab_test=ascab_val,
    observation_filter=list(ascab_train.observation_space.keys()),
    render=False,
    path_model=os.path.join(os.getcwd(), 'rl_model'),
    path_log=os.path.join(os.getcwd(), 'log'),
    rl_algorithm=DQN,  # feed in the constructor of the Stable Baselines 3 model
    seed=107,  # use random seed if you like to
)

If you don't want to use the Stable Baselines 3 framework, you can of course start training using any RL framework you prefer, starting by using the `ascab_train` gym environment defined above.

### Don't want to use RL? No problem, we got you!
It's nevertheless possible to create your own non-RL agent; a conditional agent!
One conditional agent could be a spraying schedule based on calendar date. This strategy is typically employed by farmers. Here's an example of how to do it:

### Example of creating a conditional agent

In [ ]:
from datetime import datetime
from ascab.train import BaseAgent
from ascab.env.env import AScabEnv

# First define the class, not forgetting to SubClass `BaseAgent`
# In general, you want to change the get_action method to apply your conditional spraying strategy
class HowMyLocalFarmerSprays(BaseAgent):
    def __init__(
        self,
        ascab: AScabEnv = None,
        render: bool = True,
        dates: list[datetime.date] = None
    ):
        super().__init__(ascab=ascab, render=render)
        if dates is None:
            year = self.ascab.get_wrapper_attr("date").year
            dates = [datetime.date(year, 3, 20), datetime.date(year, 4, 1), datetime.date(year, 4, 8)]  # dates you want to spray
        self.dates = dates

    def get_action(self, observation: dict = None) -> float:
        if self.ascab.get_wrapper_attr("info")["Date"] and self.ascab.get_wrapper_attr("info")["Date"][-1] in self.dates:
            return 1.0  # you spray this much if when it is the date arrives
        return 0.0


Then, you can try and run your expert strategy in the validation environment!

In [ ]:
farmer_strategy = HowMyLocalFarmerSprays(ascab_val)
farmer_strategy.run()

How did it do? Continue experimenting for the best spraying strategy!

Also, you can evaluate a trained RL agent the same way:

In [ ]:
rl_agent.run()

Example of evaluation

Example of creating MyAgent

# Instructions for submitting your winning Agent

1. Obs space is fixed; allowed features
2. Training file is fixed
3. We provide Val file
4. Testing for leaderboard will be done in unseen location